In [2]:
import argparse
import os
from epitome.models import *
from epitome.functions import *
from epitome.viz  import *

from epitome.constants import *
from epitome.motif_functions import *
import yaml
import subprocess
from timeit import default_timer as timer

/home/eecs/jahnavis/miniconda/envs/EpitomeNewEnv2/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


### Set Up

In [16]:
results_path = "results"
epitome_data_path = "data/epitome_data" 
motif_dir = "data/motif_data/"
feature_path = os.path.join(epitome_data_path, "feature_name")

# TF's being predicted
TF = "JUND"
query_cell = 'K562' #'T47D'

train_iterations = 5000
test_iterations = 10000

In [17]:
def setup_directories(TF, results_path=results_path):
    # create user directories if they do not exist
    epitome_results_dir = os.path.join(results_path, "epitome_results")
    if not os.path.exists(epitome_results_dir):
        os.makedirs(epitome_results_dir)

    epitome_models_dir = os.path.join(results_path, "epitome_models")
    if not os.path.exists(epitome_results_dir):
        os.makedirs(epitome_models_dir)

    # Folder based on TF being predicted
    tf_results_dir = os.path.join(epitome_results_dir, TF + "_results")
    if not os.path.exists(tf_results_dir):
        os.makedirs(tf_results_dir)

    # Folder based on TF being predicted
    tf_model_dir = os.path.join(epitome_models_dir, TF + "_models")
    if not os.path.exists(tf_model_dir):
        os.makedirs(tf_model_dir)
    
    return tf_results_dir, tf_model_dir

tf_results_dir, tf_model_dir = setup_directories(TF)

### Load in Data for Epitome

In [18]:
train_data = scipy.sparse.load_npz(os.path.join(epitome_data_path, 'train.npz')).toarray()
valid_data = scipy.sparse.load_npz(os.path.join(epitome_data_path, 'valid.npz')).toarray()
test_data = scipy.sparse.load_npz(os.path.join(epitome_data_path, 'test.npz')).toarray()
data = {Dataset.TRAIN: train_data, Dataset.VALID: valid_data, Dataset.TEST: test_data}
# all_data = np.concatenate((data[Dataset.TRAIN], data[Dataset.VALID], data[Dataset.TEST]), axis=1)

In [19]:
motifmat = np.load(os.path.join(motif_dir, "OVERLAP_HOCOMOCO_unique_motifmat.npz"))["tf"]
motifmap = pd.read_csv(os.path.join(motif_dir, "OVERLAP_HOCOMOCO_unique_motifmap.csv"), 
                       header=None).rename(columns={0:"Index", 1:"TF"})

In [8]:
# Determine Anchor TF's we have data for
epitome_tfs = list(motifmap["TF"].unique()) + ["DNase"]
anchor_tfs = ["CTCF", "E2F1", "EGR1", "FOXA1", "FOXA2", "GABPA", "HNF4A", "JUND", 
              "MAX", "NANOG", "REST", "TAF1"]
anchor_overlap_tfs = set(epitome_tfs).intersection(set(anchor_tfs))
len(anchor_tfs), len(anchor_overlap_tfs), anchor_overlap_tfs

(12,
 9,
 {'CTCF', 'E2F1', 'EGR1', 'FOXA1', 'GABPA', 'JUND', 'MAX', 'REST', 'TAF1'})

### Train VLP Model With Motif Data

In [21]:
matrix, cellmap, assaymap = get_assays_from_feature_file(feature_path,
                                                         eligible_assays = TF,
                                                         eligible_cells = None, 
                                                         min_cells_per_assay = 2, 
                                                         min_assays_per_cell= 2) #10)

In [22]:
model = VLP(TF,
            data = data,
            matrix = matrix,
            cellmap = cellmap,
            assaymap = assaymap,
            motifmat = motifmat, 
            motifmap = motifmap)

start = timer()
model.train(train_iterations)
end = timer()
train_time = end-start
print('epitome train time %f' % train_time)

model_path = os.path.join(tf_model_dir, query_cell + "_" + TF + "_motif")
model.save(model_path)

using ['T47D', 'SK-N-SH', 'MCF-7', 'K562', 'HepG2', 'HeLa-S3', 'HCT116', 'H1', 'GM12878', 'A549'] as labels for mode Dataset.TRAIN
using ['T47D', 'SK-N-SH', 'MCF-7', 'K562', 'HepG2', 'HeLa-S3', 'HCT116', 'H1', 'GM12878', 'A549'] as labels for mode Dataset.VALID
INFO:tensorflow:Starting Training
INFO:tensorflow:0 tf.Tensor(71.51683, shape=(), dtype=float32)tf.Tensor(32.367836, shape=(), dtype=float32)tf.Tensor(39.148994, shape=(), dtype=float32)
INFO:tensorflow:1000 tf.Tensor(43.75319, shape=(), dtype=float32)tf.Tensor(17.765388, shape=(), dtype=float32)tf.Tensor(25.9878, shape=(), dtype=float32)
INFO:tensorflow:2000 tf.Tensor(27.510347, shape=(), dtype=float32)tf.Tensor(10.572031, shape=(), dtype=float32)tf.Tensor(16.938316, shape=(), dtype=float32)
INFO:tensorflow:3000 tf.Tensor(27.68876, shape=(), dtype=float32)tf.Tensor(15.903814, shape=(), dtype=float32)tf.Tensor(11.7849455, shape=(), dtype=float32)
INFO:tensorflow:4000 tf.Tensor(24.891537, shape=(), dtype=float32)tf.Tensor(15.9134

In [23]:
model_results = model.test(test_iterations, calculate_metrics=True)
print('Model auROC: %s. Model auPRC: %s.' % (model_results['auROC'], model_results['auPRC'])) 

157it [00:39,  3.94it/s]

INFO:tensorflow:macro auROC:     0.9084975686826244
INFO:tensorflow:auPRC:     0.02531934997275975
INFO:tensorflow:GINI:     0.5623797544778633
Model auROC: 0.9084975686826244. Model auPRC: 0.02531934997275975.


In [24]:
eval_results_df = pd.DataFrame(columns=['query_cell', 'auROC', 'auPRC'])
eval_results_df = eval_results_df.append({ 
   'predicted_transcription_factor' : TF, #", ".join(anchor_overlap_tfs),
   'query_cell' : query_cell,
   'auROC' : model_results['auROC'],
   'auPRC' : model_results['auPRC'],
   'trained_transcription_factors' : TF, #", ".join(anchor_overlap_tfs),
   'iterations_trained' : train_iterations,
   'iterations_tested' : test_iterations,
   'train_time' : train_time,
    }, 
    ignore_index=True)
eval_results_df.to_csv(os.path.join(tf_results_dir, query_cell + "_" + TF + '_motif.csv'), 
                       sep="\t")

preds_file = os.path.join(tf_results_dir, query_cell + "_" + TF + '_motif.npz')
np.savez_compressed(preds_file ,pred=model_results['preds_mean'].numpy())

In [29]:
model_results['preds_mean'].numpy()

array([[0.03256017, 0.02350445],
       [0.12821168, 0.04365126],
       [0.064019  , 0.03120099],
       ...,
       [0.07639625, 0.03591267],
       [0.01666835, 0.0120222 ],
       [0.01700682, 0.00828141]], dtype=float32)

In [30]:
model_results

{'preds_mean': <tf.Tensor: shape=(10000, 2), dtype=float32, numpy=
 array([[0.03256017, 0.02350445],
        [0.12821168, 0.04365126],
        [0.064019  , 0.03120099],
        ...,
        [0.07639625, 0.03591267],
        [0.01666835, 0.0120222 ],
        [0.01700682, 0.00828141]], dtype=float32)>,
 'preds_std': <tf.Tensor: shape=(10000, 2), dtype=float32, numpy=
 array([[0.02154262, 0.01981107],
        [0.05854309, 0.02100998],
        [0.04072119, 0.02334389],
        ...,
        [0.03715775, 0.02083966],
        [0.01057944, 0.00872744],
        [0.0090576 , 0.00503344]], dtype=float32)>,
 'truth': <tf.Tensor: shape=(10000, 2), dtype=float32, numpy=
 array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]], dtype=float32)>,
 'weights': <tf.Tensor: shape=(10000, 2), dtype=float32, numpy=
 array([[1., 0.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 1.],
        [1., 0.],
        [1., 1.]], dtype=floa

### VLP Model Without Motif Data

In [26]:
matrix, cellmap, assaymap = get_assays_from_feature_file(feature_path,
                                                         eligible_assays = TF,
                                                         eligible_cells = None, 
                                                         min_cells_per_assay = 2, 
                                                         min_assays_per_cell= 2) #10)

In [27]:
model = VLP(TF,
            data = data,
            matrix = matrix,
            cellmap = cellmap,
            assaymap = assaymap)

start = timer()
model.train(train_iterations)
end = timer()
train_time = end-start
print('epitome train time %f' % train_time)

model_path = os.path.join(tf_model_dir, query_cell + "_" + TF + "_no_motif")
model.save(model_path)

using ['T47D', 'SK-N-SH', 'MCF-7', 'K562', 'HepG2', 'HeLa-S3', 'HCT116', 'H1', 'GM12878', 'A549'] as labels for mode Dataset.TRAIN
using ['T47D', 'SK-N-SH', 'MCF-7', 'K562', 'HepG2', 'HeLa-S3', 'HCT116', 'H1', 'GM12878', 'A549'] as labels for mode Dataset.VALID
INFO:tensorflow:Starting Training
INFO:tensorflow:0 tf.Tensor(59.26118, shape=(), dtype=float32)tf.Tensor(33.030785, shape=(), dtype=float32)tf.Tensor(26.230398, shape=(), dtype=float32)
INFO:tensorflow:1000 tf.Tensor(42.590797, shape=(), dtype=float32)tf.Tensor(25.06586, shape=(), dtype=float32)tf.Tensor(17.524937, shape=(), dtype=float32)
INFO:tensorflow:2000 tf.Tensor(34.080284, shape=(), dtype=float32)tf.Tensor(22.320139, shape=(), dtype=float32)tf.Tensor(11.760146, shape=(), dtype=float32)


KeyboardInterrupt: 

In [ ]:
model_results = model.test(test_iterations, calculate_metrics=True)
print('Model auROC: %s. Model auPRC: %s.' % (model_results['auROC'], model_results['auPRC'])) 

In [ ]:
eval_results_df = pd.DataFrame(columns=['query_cell', 'auROC', 'auPRC'])
eval_results_df = eval_results_df.append({ 
   'predicted_transcription_factor' : TF, #", ".join(anchor_overlap_tfs),
   'query_cell' : query_cell,
   'auROC' : model_results['auROC'],
   'auPRC' : model_results['auPRC'],
   'trained_transcription_factors' : TF, #", ".join(anchor_overlap_tfs),
   'iterations_trained' : train_iterations,
   'iterations_tested' : test_iterations,
   'train_time' : train_time,
    }, 
    ignore_index=True)
eval_results_df.to_csv(os.path.join(tf_results_dir, query_cell + "_" + TF + '_no_motif.csv'), 
                       sep="\t")

preds_file = os.path.join(tf_results_dir, query_cell + "_" + TF + '_no_motif.npz')
np.savez_compressed(preds_file ,pred=model_results['preds_mean'].numpy())

### Train/ Evaluate on all Anchor TF's

In [58]:
for tf in anchor_overlap_tfs:
    # TF's being predicted
    TF = tf
    
    if TF == "CTCF":
        continue
    
    print("Training %s..." % TF)
    query_cell = 'K562' #'T47D'
    
    tf_results_dir, tf_model_dir = setup_directories(TF)
    
    matrix, cellmap, assaymap = get_assays_from_feature_file(feature_path,
                                                         eligible_assays = TF,
                                                         eligible_cells = None, 
                                                         min_cells_per_assay = 2, 
                                                         min_assays_per_cell= 2)
    
    # Train TF with Motif Data
    model = VLP(TF,
            data = data,
            matrix = matrix,
            cellmap = cellmap,
            assaymap = assaymap,
            motifmat = motifmat, 
            motifmap = motifmap)

    start = timer()
    model.train(train_iterations)
    end = timer()
    train_time = end-start
    print('%s Motif Train Time %f' % (TF, train_time))

    model_path = os.path.join(tf_model_dir, query_cell + "_" + TF + "_motif")
    model.save(model_path)
    
    # Test TF with Motif Model
    model_results = model.test(test_iterations, calculate_metrics=True)
#     print('%s: auROC: %s. auPRC: %s.' % (TF, model_results['auROC'], model_results['auPRC'])) 
    
    eval_results_df = pd.DataFrame(columns=['query_cell', 'auROC', 'auPRC'])
    eval_results_df = eval_results_df.append({ 
       'predicted_transcription_factor' : TF,
       'query_cell' : query_cell,
       'auROC' : model_results['auROC'],
       'auPRC' : model_results['auPRC'],
       'trained_transcription_factors' : TF,
       'iterations_trained' : train_iterations,
       'iterations_tested' : test_iterations,
       'train_time' : train_time,
        }, 
        ignore_index=True)
    eval_results_df.to_csv(os.path.join(tf_results_dir, query_cell + "_" + TF + '_motif.csv'), 
                           sep="\t")

    preds_file = os.path.join(tf_results_dir, query_cell + "_" + TF + '_motif.npz')
    np.savez_compressed(preds_file ,pred=model_results['preds_mean'].numpy())
    
    # Train TF without Motif Data
    model = VLP(TF,
            data = data,
            matrix = matrix,
            cellmap = cellmap,
            assaymap = assaymap)

    start = timer()
    model.train(train_iterations)
    end = timer()
    train_time = end-start
    print('%s Non-Motif Train Time %f' % (TF, train_time))

    model_path = os.path.join(tf_model_dir, query_cell + "_" + TF + "_no_motif")
    model.save(model_path)
    
    # Test TF with Non-Motif Model
    model_results = model.test(test_iterations, calculate_metrics=True)
#     print('%s: auROC: %s. auPRC: %s.' % (TF, model_results['auROC'], model_results['auPRC'])) 
    
    eval_results_df = pd.DataFrame(columns=['query_cell', 'auROC', 'auPRC'])
    eval_results_df = eval_results_df.append({ 
       'predicted_transcription_factor' : TF,
       'query_cell' : query_cell,
       'auROC' : model_results['auROC'],
       'auPRC' : model_results['auPRC'],
       'trained_transcription_factors' : TF,
       'iterations_trained' : train_iterations,
       'iterations_tested' : test_iterations,
       'train_time' : train_time,
        }, 
        ignore_index=True)
    eval_results_df.to_csv(os.path.join(tf_results_dir, query_cell + "_" + TF + '_no_motif.csv'), 
                           sep="\t")

    preds_file = os.path.join(tf_results_dir, query_cell + "_" + TF + '_no_motif.npz')
    np.savez_compressed(preds_file ,pred=model_results['preds_mean'].numpy())

Training JUND...
using ['T47D', 'SK-N-SH', 'MCF-7', 'K562', 'HepG2', 'HeLa-S3', 'HCT116', 'H1', 'GM12878', 'A549'] as labels for mode Dataset.TRAIN
using ['T47D', 'SK-N-SH', 'MCF-7', 'K562', 'HepG2', 'HeLa-S3', 'HCT116', 'H1', 'GM12878', 'A549'] as labels for mode Dataset.VALID
INFO:tensorflow:Starting Training
INFO:tensorflow:0 tf.Tensor(67.876144, shape=(), dtype=float32)tf.Tensor(28.68024, shape=(), dtype=float32)tf.Tensor(39.195908, shape=(), dtype=float32)
INFO:tensorflow:1000 tf.Tensor(39.79464, shape=(), dtype=float32)tf.Tensor(13.760158, shape=(), dtype=float32)tf.Tensor(26.034485, shape=(), dtype=float32)
INFO:tensorflow:2000 tf.Tensor(34.20086, shape=(), dtype=float32)tf.Tensor(17.108875, shape=(), dtype=float32)tf.Tensor(17.091986, shape=(), dtype=float32)
INFO:tensorflow:3000 tf.Tensor(32.006634, shape=(), dtype=float32)tf.Tensor(19.966705, shape=(), dtype=float32)tf.Tensor(12.039928, shape=(), dtype=float32)
INFO:tensorflow:4000 tf.Tensor(25.870155, shape=(), dtype=float32

0it [00:00, ?it/s]

JUND Motif Train Time 1687.140229


157it [00:40,  3.89it/s]

INFO:tensorflow:macro auROC:     0.9284678551855514
INFO:tensorflow:auPRC:     0.012592098172470794
INFO:tensorflow:GINI:     0.7503683369564035


...

using ['K562', 'H1', 'GM12878'] as labels for mode Dataset.TRAIN
using ['K562', 'H1', 'GM12878'] as labels for mode Dataset.VALID
INFO:tensorflow:Starting Training
INFO:tensorflow:0 tf.Tensor(50.697414, shape=(), dtype=float32)tf.Tensor(43.30559, shape=(), dtype=float32)tf.Tensor(7.391823, shape=(), dtype=float32)
INFO:tensorflow:1000 tf.Tensor(24.07993, shape=(), dtype=float32)tf.Tensor(18.815435, shape=(), dtype=float32)tf.Tensor(5.264493, shape=(), dtype=float32)
INFO:tensorflow:2000 tf.Tensor(8.728676, shape=(), dtype=float32)tf.Tensor(4.8418174, shape=(), dtype=float32)tf.Tensor(3.886859, shape=(), dtype=float32)
INFO:tensorflow:3000 tf.Tensor(3.5481765, shape=(), dtype=float32)tf.Tensor(0.42921558, shape=(), dtype=float32)tf.Tensor(3.1189609, shape=(), dtype=float32)
INFO:tensorflow:4000 tf.Tensor(11.482698, shape=(), dtype=float32)tf.Tensor(8.733459, shape=(), dtype=float32)tf.Tensor(2.749239, shape=(), dtype=float32)


0it [00:00, ?it/s]

EGR1 Non-Motif Train Time 822.888214


157it [00:21,  7.38it/s]

INFO:tensorflow:macro auROC:     0.866631605601979
INFO:tensorflow:auPRC:     0.28948371621693886
INFO:tensorflow:GINI:     0.7332631740660701
